In [1]:
import json
import pyspark.sql.functions as F
from utils import get_range
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, ArrayType, IntegerType

In [2]:
CELL_AREA    = '/data/cell_area.csv'
PARQUETS_DIR = '/data/parquets'
CURRENT_DIR  = '/home/hellscream/Documents/backendSpark'

## DataFrame Schema

## Create Spark Session

In [3]:
class SparkSessionBase():
    def __init__(self):
        self.spark = SparkSession\
                     .builder\
                     .appName('Mobility')\
                     .getOrCreate()
        
        self.cell_area_df = self.spark.read.format('csv').options(header='true', delimiter='\t')\
                         .load(CURRENT_DIR + CELL_AREA)\
                         .select('id', 'area_correlator', 'latitude', 'longitude', 'province')

In [16]:
class Mobility(SparkSessionBase):
    def __init__(self, date, time_start_lower, time_start_high, time_end_lower,
        time_end_high, time_sleep_lower='01:00', time_sleep_high='04:00'):
        super().__init__()
        self.date = date
        
    def get_mobility_at_time_interval(self, time_start, time_end):
        # load correspondent parquet
        df = self.spark.read.parquet(CURRENT_DIR + PARQUETS_DIR + '/' + self.date)

        get_range_udf = F.udf(lambda elems, a, b : get_range(elems, a, b), ArrayType(IntegerType()))

        df = df.withColumn('range', get_range_udf(df.times, F.lit(time_start), F.lit(time_end)))\
               .select(df.code,\
               F.slice(df.towers, F.col('range')[0], F.col('range')[1]).alias('towers'),\
               F.slice(df.times,  F.col('range')[0], F.col('range')[1]).alias('times'))\
               .where(F.size(F.col('towers')) > 0)
        
        # df.printSchema()
        df.show(10)
        
        df = df.select('code', F.explode('towers').alias('cell'))\
               .groupBy('code', 'cell')\
               .agg(F.count('cell').alias('count'))
        
        return df

In [27]:
mobility_instance = Mobility('sample', None, None, None, None)
user_cells_start = mobility_instance.get_mobility_at_time_interval(30, 7000)
user_cells_end   = mobility_instance.get_mobility_at_time_interval(1400, 2700)

+----+--------------------+--------------------+
|code|              towers|               times|
+----+--------------------+--------------------+
| 273|[gxhip, qzijd, yj...|[78, 349, 461, 76...|
| 839|[ixdgk, lwitd, um...|[51, 67, 71, 75, ...|
| 291|[qtkbq, dfxpf, ff...|[206, 217, 412, 4...|
| 210|[xxwuj, youmx, md...|[81, 162, 191, 22...|
| 220|[lkpin, htwbq, cl...|[31, 51, 107, 129...|
| 542|[kuhds, hdqsf, en...|[54, 89, 271, 308...|
| 995|[fquzt, nrfyk, va...|[46, 49, 64, 69, ...|
| 659|[czctq, ikfbu, eg...|[69, 82, 92, 141,...|
| 681|[eqisd, daviq, lw...|[46, 97, 112, 138...|
| 501|[wgiwe, qpsft, lt...|[92, 99, 121, 122...|
+----+--------------------+--------------------+
only showing top 10 rows

+----+--------------------+--------------------+
|code|              towers|               times|
+----+--------------------+--------------------+
| 839|[vtokd, lewpi, bh...|[1402, 1412, 1427...|
| 291|             [jrzrn]|              [1476]|
| 210|[rnikk, qrurz, xv...|[1410, 1425, 144

In [6]:
# TODO: Normalize users_cells?

In [7]:
user_cells_start.show()

+----+-----+-----+
|code| cell|count|
+----+-----+-----+
| 965|qtpew|    1|
| 122|yzlfc|    1|
| 504|uwezh|    1|
| 389|hmdce|    1|
| 394|sawmv|    1|
|   7|sqjos|    1|
|   7|tydxf|    1|
|  14|phsgp|    1|
|  14|etvuh|    1|
| 467|vjwwt|    1|
| 467|tocfa|    1|
| 209|dlwbe|    1|
| 209|ehius|    1|
| 469|rcoak|    1|
| 275|ayqfs|    1|
| 871|frwrt|    1|
| 355|cdjoh|    1|
| 387|ovsnu|    1|
| 348|tpvgf|    1|
| 199|ovriz|    1|
+----+-----+-----+
only showing top 20 rows



In [8]:
user_cells_end.show()

+----+-----+-----+
|code| cell|count|
+----+-----+-----+
| 848|hwwpn|    1|
| 100|tityo|    1|
| 128|tinlu|    1|
| 654|cgzfj|    1|
| 702|ygete|    1|
|  87|ufxwn|    1|
| 742|byyjp|    1|
|1000|yuvex|    1|
| 147|qhvna|    1|
| 256|qgxys|    1|
| 304|jzpus|    1|
| 400|mzyfy|    1|
| 889|mfnas|    1|
| 361|rwhmy|    1|
| 424|dcmtf|    1|
| 361|zcvzw|    1|
| 988|enjkw|    1|
| 651|bnawj|    1|
| 284|tbkhb|    1|
| 435|geqbn|    1|
+----+-----+-----+
only showing top 20 rows



In [10]:
inter = user_cells_start.select('code')\
        .intersect(user_cells_end.select('code'))\
        .withColumnRenamed('code', 'inter')
inter.show()

+-----+
|inter|
+-----+
|   26|
|  418|
|   65|
|  191|
|  938|
|  270|
|  222|
|  278|
|  965|
|  296|
|  926|
|   19|
|  348|
|  287|
|  847|
|  415|
|  831|
|  736|
|  155|
|  112|
+-----+
only showing top 20 rows



In [28]:
# Para quedarme con la data completa, solo de los usuarios en la interseccion
# Creo que me puedo ahorrar este paso


user_cells_start = inter.join(user_cells_start)\
                        .select('code', 'cell', 'count')\
                        .filter(F.col('code') == F.col('inter'))

user_cells_end = inter.join(user_cells_end)\
                      .select('code', 'cell', 'count')\
                      .filter(F.col('code') == F.col('inter'))\
                      .withColumnRenamed('code', 'code_1')\
                      .withColumnRenamed('cell', 'cell_1')\
                      .withColumnRenamed('count', 'count_1')
            

#user_cells_start.show()
#user_cells_end.show()
users_interception = user_cells_start.join(user_cells_end)
users_interception.show()

+----+-----+-----+------+------+-------+
|code| cell|count|code_1|cell_1|count_1|
+----+-----+-----+------+------+-------+
| 965|qtpew|    1|   848| hwwpn|      1|
| 965|qtpew|    1|   100| tityo|      1|
| 965|qtpew|    1|   128| tinlu|      1|
| 965|qtpew|    1|   654| cgzfj|      1|
| 965|qtpew|    1|   702| ygete|      1|
| 965|qtpew|    1|    87| ufxwn|      1|
| 965|qtpew|    1|   742| byyjp|      1|
| 965|qtpew|    1|  1000| yuvex|      1|
| 965|qtpew|    1|   147| qhvna|      1|
| 965|qtpew|    1|   256| qgxys|      1|
| 965|qtpew|    1|   304| jzpus|      1|
| 965|qtpew|    1|   400| mzyfy|      1|
| 965|qtpew|    1|   889| mfnas|      1|
| 965|qtpew|    1|   361| rwhmy|      1|
| 965|qtpew|    1|   424| dcmtf|      1|
| 965|qtpew|    1|   361| zcvzw|      1|
| 965|qtpew|    1|   988| enjkw|      1|
| 965|qtpew|    1|   651| bnawj|      1|
| 965|qtpew|    1|   284| tbkhb|      1|
| 965|qtpew|    1|   435| geqbn|      1|
+----+-----+-----+------+------+-------+
only showing top

In [39]:
# Supuesta 'matriz'

matriz = users_interception.withColumn('val', F.col('count') * F.col('count_1'))\
                           .select('cell', 'cell_1', 'val')\
                           .filter(F.col('code') == F.col('code_1'))\
                           .groupBy('cell', 'cell_1')\
                           .agg(F.sum(F.col('val')))
matriz.show()

+-----+------+--------+
| cell|cell_1|sum(val)|
+-----+------+--------+
|uajsk| sgdgo|       1|
|rwbyi| yzumf|       1|
|anbqr| itduz|       1|
|zygcv| itduz|       1|
|skerw| bdloe|       1|
|imfqf| rrdkc|       1|
|izcfe| omrsc|       1|
|oqcds| vvikf|       1|
|jvjha| uxfke|       1|
|gxqrt| jeryo|       1|
|gxqrt| vjlpc|       1|
|rxonr| ccite|       1|
|fwdfl| ubbxv|       1|
|zcuoq| osizt|       1|
|hpkee| hhgzr|       1|
|xlyvl| mmdkm|       1|
|enkxy| emuhk|       1|
|qamwy| qidtx|       1|
|jkabw| jxmbd|       1|
|berdf| ouovs|       1|
+-----+------+--------+
only showing top 20 rows

